In [1]:
import bs4 as bs
import urllib.request
import pandas as pd
import requests
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
from datetime import datetime
import matplotlib.ticker as mtick

In [2]:
import re

def clean(string):
    retval = re.sub(r'<.+?>', '', string)
    rmv = "Have an opinion on this story? Share it!"
    retval = retval.split(rmv, 1)[0]
    retval = re.sub(r'\b.+\xa0', '', retval).replace('\n, ', '').lower()
    retval = re.sub(r'\n\n\n\n\n.+', '', retval).replace('\n', '')
    return retval

In [68]:
def SoupScraping(URL):
    column_names = ['articleid', 'url', 'title', 'content', 'sources']
    df = pd.DataFrame(columns=column_names)
    article_id = 1
    
    condition = True
    while condition:
        source = requests.get(URL).content
        soup = bs.BeautifulSoup(source, 'lxml')
        
        url_list = []
        headers = []
        for urls in soup.find_all('div', id='mw-pages'):
            url_list.extend(urls.find_all('a'))
            headers.extend(urls.find_all(re.compile(r'^h[1-6]$')))
        
        if headers[-1].text >= 'F' and headers[-1].text <= 'O':
            print("Scraping articles from:", URL)
        else: 
            condition = False
            
        URL = 'https://en.wikinews.org' + url_list[1]['href']
        for url in url_list[2:-2]:
            article_url = "https://en.wikinews.org" + url['href']
            
            # Just in case server is overloaded
            try:
                article_source = requests.get(article_url, timeout=2.5)
            except:
                print("Connection timed out - Retrying request...")
                article_source = requests.get(article_url, timeout=2.5)
                
            content = article_source.content
            article_soup = bs.BeautifulSoup(content, 'lxml')
            
            article_title = article_soup.find(class_='firstHeading').text
            
            text_list = []
            for text in article_soup.find_all(class_='mw-parser-output'):
                text_list.extend(text.find_all('p'))
            
            text = clean(str(text_list))
            
            sources_list = []
            for source in article_soup.find_all('a', rel='nofollow', class_='external text'):
                sources_list.append(source['href'])
            
            df = df.append({'articleid':article_id, 'url':article_url, 'title':article_title
                            , 'content':text, 'sources':sources_list},
                            ignore_index=True)
            article_id += 1
    return df

In [69]:
group_nr = 5
urls = []
for char in "ABCDEFGHIJKLMNOPRSTUVWZABCDEFGHIJKLMNOPRSTUVWZ"[group_nr%23:group_nr%23+10]:
    urls.append('https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=' + char)


In [70]:
def retSoup():
    return SoupScraping('https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=F')

Scraping articles from: https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=F
Scraping articles from: https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&pagefrom=Former+Managing+Director+of+Gambian+newspaper+appears+in+court&subcatfrom=F&filefrom=F#mw-pages
Scraping articles from: https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&pagefrom=Gaddafi+loyalists+allegedly+using+Red+Cross+helicopters+to+bomb+rebel+held+city&subcatfrom=F&filefrom=F#mw-pages
Scraping articles from: https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&pagefrom=Gujarat+high+court+refutes+Modi%27s+objection+to+Lokyukta%27s+appointment&subcatfrom=F&filefrom=F#mw-pages
Scraping articles from: https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&pagefrom=Illinois+Governor+Blagojevich+appoints+Roland+Burris+for+vacated+U.S.+Senate+seat&subcatfrom=F&filefrom=F#mw-pages
Scraping articles from: https://en.w

In [71]:
df

,articleid,url,title,content,sources
0,1,https://en.wikinews.org/wiki/Face_coverings_to...,Face coverings to be mandatory on public trans...,"[, yesterday, england's transport secretary, g...","[https://www.bbc.co.uk/news/uk-52927089, https..."
1,2,https://en.wikinews.org/wiki/Face-off_at_Islam...,Face-off at Islamabad Mosque,"[since last tuesday, following violent confron...",[http://news.bbc.co.uk/1/hi/world/south_asia/6...
2,3,https://en.wikinews.org/wiki/Facebook_blocked_...,Facebook blocked in Bangladesh,[bangladesh temporarily blocked the social net...,[http://www.thedailystar.net/newDesign/news-de...
3,4,https://en.wikinews.org/wiki/Facebook_takes_do...,Facebook takes down groups supporting Austin c...,[on the popular social networking website face...,[http://www.azcentral.com/news/articles/2010/0...
4,5,https://en.wikinews.org/wiki/Failings_identifi...,Failings identified in response to Iranian sei...,[two inquiries into the seizure of royal navy ...,[http://news.bbc.co.uk/1/hi/uk_politics/676590...
...,...,...,...,...,...
2795,2796,https://en.wikinews.org/wiki/Photo_essay:_Vale...,Photo essay: Valentine's Day at the U.S. Viet ...,"[washington, d.c. —the vietnam veterans memori...",[]
2796,2797,"https://en.wikinews.org/wiki/Pickens_County,_S...","Pickens County, South Carolina sheriff refuses...","[on friday, the sheriff of pickens county, sou...",[http://www.foxnews.com/politics/2013/12/08/sc...
2797,2798,https://en.wikinews.org/wiki/Pilot_with_9/11_l...,Pilot with 9/11 links deported from New Zealand,"[a yemeni man linked to the september 11, 2001...",[https://www.beehive.govt.nz/ViewDocument.aspx...
2798,2799,https://en.wikinews.org/wiki/Pirate_attacks_br...,Pirate attacks bring UN aid to Somalia to a halt,"[on saturday, may 19, a ship chartered by the ...",[http://www.voanews.com/english/Africa/2007-05...


In [ ]:
formats = ["%B %d %Y", "%d %B %Y", "%b %d %Y", "%m/%d/%Y", "%m %d %Y"]
published = df['published']

all_dates = []
for publish in published:
    if publish != 'None':
        date = re.sub(r'^.*?, ', '', publish).replace(',','')

        for format in formats:
            try:
                all_dates.append(datetime.strptime(date, format).strftime("%Y-%m-%d"))
            except ValueError:
            pass
sorted_dates = sorted(all_dates)
pd_sorted_dates = pd.DataFrame(sorted_dates)

In [74]:
#CREATE TABLE FOR SCRAPED ARTICLES
'''
CREATE TABLE IF NOT EXISTS Webarticle (
      articleID INT NOT NULL PRIMARY KEY,
      url VARCHAR NOT NULL UNIQUE,
      title VARCHAR NULL,
      webcontent VARCHAR NULL,
      sources VARCHAR NULL
    );
'''
# CREATE VIEW 
'''
CREATE VIEW [article vs webarticle] AS SELECT 
    a.articleID, a.title, a.content, a.inserted_at, 
    w.webarticleID, w.webtitle, w.webcontent, w.published
    FROM article a, webarticle w;
'''

'\nCREATE TABLE IF NOT EXISTS Webarticle (\n      articleID INT NOT NULL PRIMARY KEY,\n      url VARCHAR NOT NULL UNIQUE,\n      title VARCHAR NULL,\n      webcontent VARCHAR NULL,\n      sources VARCHAR NULL\n    );\n'

In [75]:
import pandas as pd 
import psycopg2, time, re

# Make connection to database
connection = psycopg2.connect(
    user = "postgres",
    password = "2ybftbchx7bfgkv",
    host = "localhost",
    port = "5432",
    database = "DataScience")
cursor = connection.cursor()

def insertTable(cols, vals, target):
        try: 
            sql = "INSERT INTO "+target+" (" +cols + ") VALUES (" + "%s,"*(len(vals.iloc[0])-1) + "%s)"
            cursor.executemany(sql, vals.values.tolist())
            connection.commit()
        except Exception as e:
            print("Something went wrong with: %s %s" % (target, str(e)))


article = df.iloc[:,[0,1,2,3,4]]
articleval = "articleID, url, title, webcontent, sources"
insertTable(articleval, article, "webarticle")